<a href="https://colab.research.google.com/github/ktpeters/Calculating-Vegetation-Indices-from-Planet-API/blob/main/Landsat_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install earthengine-api

In [ ]:
import ee
import os
import time

In [ ]:
# # Get the path to the credentials file
credentials_path = os.path.expanduser("~/.config/earthengine/credentials")

 # Remove the credentials file if it exists
if os.path.exists(credentials_path):
     os.remove(credentials_path)
     print("Earth Engine credentials removed. You are now unauthenticated.")
else:
     print("No Earth Engine credentials found.")

No Earth Engine credentials found.


In [ ]:
# Trigger the authentication flow. You only need to do this once.
ee.Authenticate()
# Initialize the library.
ee.Initialize(project='ee-covercropproject')

In [ ]:
# Define the FeatureCollection
state_abb = 'MO'
polygons = ee.FeatureCollection(f"projects/ee-covercropproject/assets/CLU_{state_abb}")

# Sort the FeatureCollection by a specific property before converting to a list
sortedCollection = polygons.sort('ID')
featureList = sortedCollection.toList(sortedCollection.size())

In [ ]:
# Define a function to mask clouds, cloud shadows, and snow/ice using the QA_PIXEL band
def mask_clouds(image):
    # Select the QA_PIXEL band
    qa = image.select('QA_PIXEL')

    # Define bitmasks for clouds, cloud shadows, and snow/ice
    cloud_bitmask = 1 << 3
    shadow_bitmask = 1 << 4
    snow_bitmask = 1 << 5

    # Create a mask for clear conditions (no clouds, no shadows, no snow/ice)
    cloud_mask = qa.bitwiseAnd(cloud_bitmask).eq(0)  # No cloud
    shadow_mask = qa.bitwiseAnd(shadow_bitmask).eq(0)  # No cloud shadow
    snow_mask = qa.bitwiseAnd(snow_bitmask).eq(0)  # No snow/ice

    # Combine the masks
    final_mask = cloud_mask.And(shadow_mask).And(snow_mask)

    # Apply the mask to the image
    return image.updateMask(final_mask)

# Function to calculate various indices and add date
def addNDVInDate(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    tvi = image.expression(
        '(pow((((B8-B4)/(B8+B4)) +(1/2)),(1/2)))*100', {
            'B8': image.select('SR_B5'),
            'B4': image.select('SR_B4')
        }).rename('TVI')
    evi = image.expression(
        '2.5* ( (B8-B4)/ (B8+ 6*B4 - 7.5*B2 +1))', {
            'B4': image.select('SR_B4'),
            'B8': image.select('SR_B5'),
            'B2': image.select('SR_B2')
        }).rename('EVI')
    satvi = image.expression(
        '((B11-B4)/(B11+B4+1)) * (1+1) - (B12/2)', {
            'B11': image.select('SR_B6'),
            'B4': image.select('SR_B4'),
            'B12': image.select('SR_B7')
        }).rename('SATVI')
    savi = image.expression(
        '((B8-B4) + (1+0.5))/ (B8-B4+0.5)', {
            'B4': image.select('SR_B4'),
            'B8': image.select('SR_B5')
        }).rename('SAVI')
    msi = image.expression(
        '(B11/B8)', {
            'B11': image.select('SR_B6'),
            'B8': image.select('SR_B5')
        }).rename('MSI')
    gndvi = image.normalizedDifference(['SR_B5', 'SR_B3']).rename('GNDVI')
    grvi = image.normalizedDifference(['SR_B3', 'SR_B4']).rename('GRVI')
    lswi = image.normalizedDifference(['SR_B5', 'SR_B6']).rename('LSWI')
    msavi2 = image.expression(
        '(2 * B5 + 1 - sqrt(pow((2 * B5 + 1), 2) - 8 * (B5 - B4)) ) / 2', {
            'B5': image.select('SR_B5'),
            'B4': image.select('SR_B4')
        }).rename('MSAVI2')
    wdvi = image.expression(
        'B8- 0.5*B4', {
            'B4': image.select('SR_B4'),
            'B8': image.select('SR_B5')
        }).rename('WDVI')
    bi = image.expression(
        'sqrt((B4*B4) +(B3*B3))/2', {
            'B4': image.select('SR_B4'),
            'B3': image.select('SR_B3')
        }).rename('BI')
    bi2 = image.expression(
        'sqrt((B4*B4) +(B3*B3)+(B8*B8))/3', {
            'B4': image.select('SR_B4'),
            'B3': image.select('SR_B3'),
            'B8': image.select('SR_B5')
        }).rename('BI2')
    ri = image.expression(
        '(B4*B4)/(B3*B3*B3)', {
            'B4': image.select('SR_B4'),
            'B3': image.select('SR_B3')
        }).rename('RI')
    ci = image.normalizedDifference(['SR_B4', 'SR_B3']).rename('CI')
    v = image.expression(
        '(B8/B4)', {
            'B8': image.select('SR_B5'),
            'B4': image.select('SR_B4')
        }).rename('V')
    ndwi = image.normalizedDifference(['SR_B5', 'SR_B6']).rename('NDWI')
    nbr = image.normalizedDifference(['SR_B5', 'SR_B7']).rename('NBR')

    newBands = ee.Image([ndvi, tvi, evi, satvi, savi, msi, gndvi, grvi, lswi,
                         msavi2, wdvi, bi, bi2, ri, ci, v, ndwi, nbr])
    return image.addBands(newBands)

# Function to reduce the regions and calculate statistics
def reduceRegionsFunction(image):
    stats = image.reduceRegions(
        collection=subsetFC,
        reducer=ee.Reducer.mean(),
        scale=30
    )
    stats_filtered = stats.filter(ee.Filter.notNull(['NDVI']))
    return stats_filtered.map(lambda feature: feature.set('date', image.get('system:time_start')).setGeometry(None))

In [ ]:
# Iterate through the FeatureCollection in chunks
#time.sleep(15000)
stepsize=1000
for i in range(0, sortedCollection.size().getInfo(), stepsize):
    subset = featureList.slice(i, i+stepsize)
    subsetFC = ee.FeatureCollection(subset)
    convexHull = subsetFC.geometry().convexHull().buffer(50)

    # Download NDVI data for each year
    for yr in range(2015, 2025):
        startDate = f'{yr-1}-05-15'
        endDate = f'{yr}-5-15'
        fileName = f'NDVI_{yr-1}_{i}'

        landsat = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
                 .filterBounds(convexHull)
                 .filterDate(startDate, endDate)
                 .map(mask_clouds)
                 .map(addNDVInDate))

        reduced = landsat.map(reduceRegionsFunction).flatten()

        # Remove geometry before exporting
        reduced_no_geom = reduced.map(lambda feature: feature.setGeometry(None))

        # Export the data to Google Drive
        task = ee.batch.Export.table.toDrive(
            collection=reduced_no_geom,
            description=fileName,
            folder=f'Landsat8_CLU_{state_abb}_sorted',
            fileFormat='CSV'
        )
        task.start()

EEException: Too many tasks already in the queue (3000, limit 3000).

In [ ]:
### To cancel tasks
# # Get a list of all tasks
# tasks = ee.batch.Task.list()
#
# # Iterate through the tasks and cancel them
# for task in tasks:
#     task_status = task.status()
#     print(task)
#     if (task_status['state'] in ['RUNNING', 'READY']) and (task_status['description'].startswith('NDVI_2017')):
#         task.cancel()
#         print(f"Cancelled task {task.id}: {task_status['description']}")

<Task OOSZIBEX2Y4IP6FLOKI5ZK5X EXPORT_FEATURES: NDVI_2017_410000 (CANCELLED)>
<Task BFNGQA6JLXNIYQQFWBH5ZFNE EXPORT_FEATURES: NDVI_2016_410000 (CANCELLED)>
<Task OJC35SVA4XQDPVWJKRUT75OA EXPORT_FEATURES: NDVI_2015_410000 (CANCELLED)>
<Task ZRDVID3H7RK43V7ZCUQ3INK2 EXPORT_FEATURES: NDVI_2014_410000 (CANCELLED)>
<Task OAMFUMYKCO6246F66ULBMMMP EXPORT_FEATURES: NDVI_2023_409000 (READY)>
Cancelled task OAMFUMYKCO6246F66ULBMMMP: NDVI_2023_409000
<Task PVH4KFYK6ILHLFZP3VAUKN5B EXPORT_FEATURES: NDVI_2022_409000 (READY)>
Cancelled task PVH4KFYK6ILHLFZP3VAUKN5B: NDVI_2022_409000
<Task FTJJKQQTQMOXOYHXHLLZ6YHG EXPORT_FEATURES: NDVI_2021_409000 (READY)>
Cancelled task FTJJKQQTQMOXOYHXHLLZ6YHG: NDVI_2021_409000
<Task SS53INICNLVACJGG2VH5JHWG EXPORT_FEATURES: NDVI_2020_409000 (READY)>
Cancelled task SS53INICNLVACJGG2VH5JHWG: NDVI_2020_409000
<Task M6CHSILC3PMIINOZLILZWV2M EXPORT_FEATURES: NDVI_2019_409000 (READY)>
Cancelled task M6CHSILC3PMIINOZLILZWV2M: NDVI_2019_409000
<Task CV2ODYAYQZANDFW6P7ZLFF

KeyboardInterrupt: 